In [1]:
import geopandas as gpd
import pandas as pd
import requests
import json

In [2]:
# Load list of current AOI regions
url = 'https://raw.githubusercontent.com/GeoscienceAustralia/dea-ard-scene-select/master/scene_select/data/Australian_AOI.json'
region_dict = json.loads(requests.get(url).text)


In [3]:
# Load Territorial Sea Zone shapefile
# Available from https://pacificdata.org/data/dataset/australia-territorial-sea-12-nautical-mile
tsz_gdf = gpd.read_file("au_tsz_line_april2022.zip").to_crs("EPSG:4326").dissolve()

## Landsat

In [4]:
# Load Landsat WRS2 tiles
landsat_gdf = gpd.read_file('https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip').to_crs("EPSG:4326")

In [5]:
# Find polygons that intersect with TSZ
landsat_tsz_gdf = gpd.sjoin(landsat_gdf, tsz_gdf, predicate='intersects')

In [6]:
# Remove any regions already in the DEA AOI and restrict to non-Antartic
landsat_tsz_nonaoi_gdf = landsat_tsz_gdf.loc[
    (~landsat_tsz_gdf.WRSPR.isin(region_dict["ls"])) & (landsat_tsz_gdf.geometry.centroid.y > -60)
]

/tmp/ipykernel_694/706837659.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (~landsat_tsz_gdf.WRSPR.isin(region_dict["ls"])) & (landsat_tsz_gdf.geometry.centroid.y > -60)


In [7]:
landsat_tsz_nonaoi_gdf.to_file("landsat_additional_regions.geojson")

In [22]:
# New Landsat regions
print(list(landsat_tsz_nonaoi_gdf.WRSPR))

['102067', '134097', '084082', '116075', '107068', '107069', '123067', '089072', '089087', '089088', '089091', '098066', '080080', '080098', '080099', '112069', '112070', '128068', '128069', '135097', '085080', '085081', '085082', '101069', '101084', '099066', '090091', '106067', '106084', '122067', '122068', '088074', '088075', '088076', '088079', '097086', '113084', '079080', '079098', '079099', '104067', '104083', '136097', '111085']


## Sentinel-2

In [8]:
# Get Sentinel-2 MRGS tile grid
!git clone https://github.com/justinelliotmeyers/Sentinel-2-Shapefile-Index.git

fatal: destination path 'Sentinel-2-Shapefile-Index' already exists and is not an empty directory.


In [9]:
# Load Sentinel-2 MRGS tiles
s2_gdf = gpd.read_file('Sentinel-2-Shapefile-Index/sentinel_2_index_shapefile.shp').to_crs("EPSG:4326")

In [10]:
# Find polygons that intersect with TSZ
s2_tsz_gdf = gpd.sjoin(s2_gdf, tsz_gdf, predicate='intersects')

In [11]:
# Remove any regions already in the DEA AOI and restrict to non-Antartic
s2_tsz_nonaoi_gdf = s2_tsz_gdf.loc[
   (~s2_tsz_gdf.Name.isin(region_dict["s2"])) & (s2_tsz_gdf.geometry.centroid.y > -60)
]

/tmp/ipykernel_694/3912617125.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (~s2_tsz_gdf.Name.isin(region_dict["s2"])) & (s2_tsz_gdf.geometry.centroid.y > -60)


In [12]:
s2_tsz_nonaoi_gdf.to_file("s2_additional_regions.geojson")

In [19]:
# New S2 regions
print(list(s2_tsz_nonaoi_gdf.Name))

['42FYF', '42FYG', '43FCA', '43FCB', '43FDA', '43FDB', '46LHM', '46LHN', '47LKG', '47LKH', '48LWP', '48LXP', '49JFL', '49KGS', '50HKG', '50HKH', '50HKJ', '50HMF', '50HNF', '50HPF', '50JKL', '50KKC', '51LVD', '51LWD', '51LWF', '51LXE', '51LYF', '51LZF', '52HBJ', '52HEK', '52HFK', '52LBL', '52LCL', '52LDK', '52LEM', '52LEN', '52LFP', '52LHP', '53HKD', '53HLC', '53HNA', '53LLJ', '53LPJ', '53LQH', '53LQJ', '53LRC', '54GYU', '54HXB', '54LTH', '54LZR', '55GBP', '55GFM', '55HGT', '55KEA', '55KFV', '55KGA', '55KHA', '55KHT', '55KHU', '55LBF', '55LBG', '55LBH', '55LBJ', '55LBK', '55LDC', '56HKC', '56HKD', '56HKE', '56HLF', '56HNK', '56KKC', '56KLC', '56KMC', '56KMV', '56KNU', '56KPA', '56KQA', '56KQB', '56KQF', '56KQG', '56KQV', '56KRB', '56KRF', '56KRG', '56LKH', '57FVA', '57FVU', '57FVV', '57FWA', '57FWU', '57FWV', '57HVE', '57HWE', '57JVF', '57JVG', '57JVH', '57JWF', '57JWG', '57JWH', '58JGN', '58JGP', '59JKH', '59JKJ']
